In [94]:
from lxml import etree

tree = etree.parse("replace.bpmn")

root = tree.getroot()
root

<Element {http://www.omg.org/spec/BPMN/20100524/MODEL}definitions at 0x215a3360c80>

In [51]:
root.nsmap

{'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL',
 'bpmndi': 'http://www.omg.org/spec/BPMN/20100524/DI',
 'dc': 'http://www.omg.org/spec/DD/20100524/DC',
 'di': 'http://www.omg.org/spec/DD/20100524/DI'}

In [77]:
tasks = root.xpath("bpmn:process/bpmn:task", namespaces=root.nsmap)

for task in tasks:
    print(task)
    print(task.attrib)

<Element {http://www.omg.org/spec/BPMN/20100524/MODEL}task at 0x215a323f2c0>
{'id': 'Activity_0m4dehc', 'name': 'A'}
<Element {http://www.omg.org/spec/BPMN/20100524/MODEL}task at 0x215a323e100>
{'id': 'Activity_06j4hty', 'name': 'B'}
<Element {http://www.omg.org/spec/BPMN/20100524/MODEL}task at 0x215a323f3c0>
{'id': 'Activity_15yx4fq', 'name': 'C'}


In [53]:
sFs = root.xpath("bpmn:process/bpmn:sequenceFlow", namespaces=root.nsmap)

for sF in sFs:
    print(sF.attrib)

{'id': 'Flow_1hi3vm0', 'sourceRef': 'StartEvent_0q4gktu', 'targetRef': 'Activity_0m4dehc'}
{'id': 'Flow_1dw8aov', 'sourceRef': 'Activity_15yx4fq', 'targetRef': 'Event_0ogpkq3'}


In [5]:
def get_left_most_sequence_flow(root, left_most_id: str) -> str:
    return root.xpath(f"bpmn:process/bpmn:task[@id='{left_most_id}']/bpmn:incoming", namespaces=root.nsmap)[0].text

def get_right_most_sequence_flow(root, right_most_id: str) -> str:
    return root.xpath(f"bpmn:process/bpmn:task[@id='{right_most_id}']/bpmn:outgoing", namespaces=root.nsmap)[0].text

In [6]:
get_left_most_sequence_flow(root, "Activity_0m4dehc")

'Flow_1hi3vm0'

In [7]:
get_right_most_sequence_flow(root, "Activity_15yx4fq")

'Flow_1dw8aov'

In [8]:
def remove_merged_tasks(root, merged_tasks: list[str]):
    bpmn_process = root.xpath("bpmn:process", namespaces=root.nsmap)[0]
    for child in bpmn_process.iter():
        if "task" in child.tag and child.attrib["id"] in merged_tasks:
            bpmn_process.remove(child)

In [43]:
def remove_tasks_in_diagram(root, tasks):
    bpmn_plane = root.xpath("bpmndi:BPMNDiagram/bpmndi:BPMNPlane", namespaces=root.nsmap)[0]
    childs_to_remove = []
    harvested_bounds = []
    for child in bpmn_plane.iter():
        if "BPMNShape" in child.tag and child.attrib["bpmnElement"] in tasks:
            childs_to_remove.append(child)
            harvested_bounds.append(child.xpath("dc:Bounds", namespaces=root.nsmap)[0].attrib)
    
    for child in childs_to_remove:
        bpmn_plane.remove(child)
        
    return harvested_bounds

In [25]:
bpmn_plane = root.xpath("bpmndi:BPMNDiagram/bpmndi:BPMNPlane", namespaces=root.nsmap)[0]
bpmn_plane

for child in bpmn_plane.iter():
    if "BPMNShape" in child.tag:

        print(child.xpath("dc:Bounds", namespaces=root.nsmap)[0].attrib)

{'x': '152', 'y': '102', 'width': '36', 'height': '36'}
{'x': '240', 'y': '80', 'width': '100', 'height': '80'}
{'x': '400', 'y': '80', 'width': '100', 'height': '80'}
{'x': '560', 'y': '80', 'width': '100', 'height': '80'}
{'x': '722', 'y': '102', 'width': '36', 'height': '36'}


In [48]:
remove_tasks_in_diagram(root, tasks=["Activity_0m4dehc", "Activity_06j4hty", "Activity_15yx4fq"])

[{'x': '240', 'y': '80', 'width': '100', 'height': '80'},
 {'x': '400', 'y': '80', 'width': '100', 'height': '80'},
 {'x': '560', 'y': '80', 'width': '100', 'height': '80'}]

In [45]:
remove_merged_tasks(root, ["Activity_0m4dehc", "Activity_06j4hty", "Activity_15yx4fq"])

In [10]:
def remove_hanging_sequence_flows(root, sequence_flows: list[str]):
    bpmn_process = root.xpath("bpmn:process", namespaces=root.nsmap)[0]
    for child in bpmn_process.iter():
        if "sequenceFlow" in child.tag and child.attrib["id"] in sequence_flows:
            bpmn_process.remove(child)

In [11]:
remove_hanging_sequence_flows(root, ["Flow_0bq7cco", "Flow_1g9jqqx"])

In [95]:
import string
import random

def generate_random_string(length: int) -> str:
    characters = string.ascii_lowercase + string.digits
    return ''.join(random.choices(characters, k=length))

def add_task(root, task_name, incoming_flow, outgoing_flow):
    bpmn_process = root.xpath("bpmn:process", namespaces=root.nsmap)[0]
    new_task = etree.SubElement(bpmn_process, f"{{{root.nsmap['bpmn']}}}task", nsmap=root.nsmap)
    new_task.attrib["id"] = f"Activity_{generate_random_string(length=6)}"
    new_task.attrib["name"] = task_name
    incoming = etree.SubElement(new_task, f"{{{root.nsmap['bpmn']}}}incoming", nsmap=root.nsmap)
    incoming.text = incoming_flow
    outgoing = etree.SubElement(new_task, f"{{{root.nsmap['bpmn']}}}outgoing", nsmap=root.nsmap)
    outgoing.text = outgoing_flow

    sf_left = bpmn_process.xpath(f"bpmn:sequenceFlow[@id='{incoming_flow}']", namespaces=root.nsmap)[0]
    sf_left.attrib["targetRef"] = new_task.attrib["id"]

    sf_right = bpmn_process.xpath(f"bpmn:sequenceFlow[@id='{outgoing_flow}']", namespaces=root.nsmap)[0]
    sf_right.attrib["sourceRef"] = new_task.attrib["id"]



In [96]:
add_task(root, "ABC", "Flow_1hi3vm0", "Flow_1dw8aov")

In [97]:
tree.write("test", pretty_print=True)